# 斑点检测
在计算机视觉中，斑点检测是指在数字图像中找出和周围区域特性不同的区域，这些特性包括光照或颜色等。一般图像中斑点区域的像素特性相似甚至相同，某种程度而言，斑点块中所有点是相似的。如果将兴趣点的特性形式化表达为像素位置的函数，那么主要有两类斑点检测方法:
- 差分方法。这类方法主要基于函数在对应像素点处的导数。
- 局部极值方法。这类方法主要是在找出函数的局部极值。

在该领域中，斑点检测也被称为兴趣点检测或者兴趣区域检测。研究斑点检测的目的有几点。其中最主要的是斑点检测能够完整的刻画像素所在区域信息，而这些信息是边缘检测和焦点检测所不能获得的。斑点检测获得的局部区域特征可以被用于后续进一步处理，比如在目标出现和部分遮挡的目标识别或追踪问题中准确刻画目标信息等。在其他领域，斑点检测还可以通过对直方图的尖峰检测实现分割。斑点检测子的另一个常见的应用是作为文本分析和识别中的基元。最新研究中，斑点检测越来越多的被应作图像宽基线立体匹配中的兴趣点检测和基于表观的目标识别中的局部图像特征。

斑点通常是指与周围有着颜色和灰度差别的区域。在实际地图中，往往存在着大量这样的斑点，如一颗树是一个斑点，一块草地是一个斑点，一栋房子也可以是一个斑点。由于斑点代表的是一个区域，相比单纯的角点，它的稳定性要好，抗噪声能力要强，所以它在图像配准上扮演了很重要的角色。同时有时图像中的斑点也是我们关心的区域，比如在医学与生物领域，我们需要从一些X光照片或细胞显微照片中提取一些具有特殊意义的斑点的位置或数量。比如下图中天空的飞机、向日葵的花盘、X线断层图像中的两个斑点

##3.4 斑点Blob
它使用拉普拉斯梯度，也就是一阶导极值点，二阶导零点。斑点对噪声很敏感，需要先做高斯平滑。先做高斯滤波，然后在做拉普拉斯滤波(也就是高斯滤波的二阶导)，和直接对高斯滤波做二次导是一样的

$\nabla^2[f(x,y) \otimes G(x,y) = \nabla^2G(x,y) \otimes f(x,y)]$

后面的叫做LoG

捕捉斑点，找拉普拉斯的极值，捕捉边缘，找拉普拉斯的临值(过0点)

LoG、DoG、DoH、SIFT、SURF